In [1]:
# Computes stochastic closure certificate using SOS for stochastic systems in verifying LTL specifications

# include important libraries
using JuMP
using MosekTools
using DynamicPolynomials
using MultivariatePolynomials
using LinearAlgebra
using TSSOS # important for SOS, see https://github.com/wangjie212/TSSOS
using Distributions # for the noise

In [2]:
@polyvar x y x0 # global vars used in monomials

sos_tol = 1 # the maximum degree of unknown SOS polynomials = deg + sos_tol 
error = 2   # precision digit places
gamma = 4.2
lamda = 130
# p-sat = 96.77%

tau1, tau2, delta = .1, 1.5, 8 # S-procedure constants in condition 3

# noise generator
function noise()
    rand() < 0.51 ? 1 : -1 # returns 1 with 51% chance, -1 with 49% chance
end

# creates the parametrized SCC
function add_paramC_poly!(model, var1, deg, q, p)
    basis = monomials(var1, 0:deg) # basis in x and y
    coeffs = @variable(model, [1:length(basis)], base_name="c_$(q)_$(p)")
    C_qp = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return C_qp, coeffs, basis
end

# creates the parametrized ranking functions
function add_paramV_poly!(model, var2, deg, q, p)
    basis = monomials(var2, 0:deg)
    coeffs = @variable(model, [1:length(basis)], base_name="v_$(q)_$(p)")
    V_qp = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return V_qp, coeffs, basis
end

# considering that AP = {{a}, {b}}
# we partition the state set based on the piecewise definition of the dynamics
fxa, fxb = 0, x + noise()
Efxa, Efxb = 0, x + 2*(0.51) - 1
fya, fyb = 0, y + noise()
Efya, Efyb = 0, y + 2*(0.51) - 1

ϵ = 1e-6

g = [-1 + 0*x] # polynomial indicating empty domain i.e -1>=0 is empty
# the domain is (label dom) n (function pairwise dom)
g_a1a, g_a2a = g, g # poly describing x==0, where L(x) = a but # [10,inf] n {x=0} = [] 
g_a1b, g_a2b = [x-10], [x-10, y-10] # [10,inf] n (0, inf] = [10,inf]
g_b1a, g_b2a = g, g # for region x>0, L(x) = b but # {x>0} n {x=0} = [] 
g_b1b, g_b2b = [x-ϵ], [x-ϵ, y-ϵ] # {x>0} n (0, inf] = (0, inf]

# for the first condition (x)
g1a = Dict(
    :"a" => g_a1a, :"b" => g_b1a
)
g1b = Dict(
    :"a" => g_a1b, :"b" => g_b1b
)
# for the 2nd condition
g2a = Dict(
    :"a" => g_a2a, :"b" => g_b2a
) 
g2b = Dict(
    :"a" => g_a2b, :"b" => g_b2b
)  
# g(x0,x,y) for 3rd condition X0=[10]
g3 = [x0-50, 50-x0]


# polynomial stochastic closure certificate of degree deg and NBA ~S having Q0 and QF
function scc(deg, S, Q0, QF)
    # synthesize SCC by using the standard formulation
    # deg: degree of scc template
    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)

    C_dict = Dict()  # key => (symbolic_poly, numeric_poly)
    
    for q in keys(S)
        for (lab_set, qn) in S[q]
            for lab in lab_set
                C_a, Cc_a, Cb_a = add_paramC_poly!(model, [x, y], deg, q, qn) # CC, CC coefficients, and CC basis functions
                # these modification takes care of the expectation
                EC_a = C_a([x, y]=>[x, Efxa]) 
                # see TSSOS https://github.com/wangjie212/TSSOS
                key_a = (q, qn, lab, "0")  
                C_dict[key_a] = (C_a, Cc_a, Cb_a)  # store symbolic, coefficients, basis

                # 1st condition
                add_psatz!(model, -EC_a + gamma, [x], g1a[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                # second part of the piecewise dyn
                C_b, Cc_b, Cb_b = add_paramC_poly!(model, [x, y], deg, q, qn) # CC, CC coefficients, and CC basis functions
                # these modification takes care of the expectation
                EC_b = C_b([x, y]=>[x, Efxb]) 
                # see TSSOS https://github.com/wangjie212/TSSOS
                key_b = (q, qn, lab, "x+w")  
                C_dict[key_b] = (C_b, Cc_b, Cb_b)  # store symbolic, coefficients, basis

                # 1st condition
                add_psatz!(model, -EC_b + gamma, [x], g1b[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                
                
                for p in keys(S)
                    C1_a, Cc1_a, Cb1_a = add_paramC_poly!(model, [x, y], deg, p, qn)
                    EC1_a = C1_a([x, y]=>[x, Efya]) 
                    C2_a, Cc2_a, Cb2_a = add_paramC_poly!(model, [x, y], deg, p, q)
                    key1_a, key2_a = (p, qn), (p, q)  
                    C_dict[key1_a], C_dict[key2_a] = (C1_a, Cc1_a, Cb1_a), (C2_a, Cc2_a, Cb2_a) 
                    
                    # 2nd condition
                    add_psatz!(model, -C1_a + C2_a, [x, y], g2a[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                    # second part of the piecewise dyn
                    C1_b, Cc1_b, Cb1_b = add_paramC_poly!(model, [x, y], deg, p, qn)
                    EC1_b = C1_b([x, y]=>[x, Efyb]) 
                    C2_b, Cc2_b, Cb2_b = add_paramC_poly!(model, [x, y], deg, p, q)
                    key1_b, key2_b = (p, qn), (p, q)  
                    C_dict[key1_b], C_dict[key2_b] = (C1_b, Cc1_b, Cb1_b), (C2_b, Cc2_b, Cb2_b) 
                    
                    # 2nd condition
                    add_psatz!(model, -C1_b + C2_b, [x, y], g2b[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                end
            end
        end
    end
    for q0 in Q0
        for qf1 in QF
            V1, Vc1, Vb1 = add_paramV_poly!(model, [x0, y], deg, q0, qf1)
            key3a = (q0, qf1, "V")  
            C_dict[key3a] = (V1, Vc1, Vb1)
            for qf2 in QF
                V2, Vc2, Vb2 = add_paramV_poly!(model, [x0, x], deg, q0, qf2)
                C3, Cc3, Cb3 = add_paramC_poly!(model, [x0, x], deg, q0, qf2)
                C4, Cc4, Cb4 = add_paramC_poly!(model, [x, y], deg, qf2, qf1)
                
                key3b = (q0, qf2, "V")  
                C_dict[key3b] = (V2, Vc2, Vb2)
                key4 = (q0, qf2)  
                C_dict[key4] = (C3, Cc3, Cb3)
                key5 = (qf1, qf2)  
                C_dict[key5] = (C4, Cc4, Cb4)
                # 3rd condition
                add_psatz!(model, -V1 + V2 - delta + tau1 * (lamda - C3) + tau2 * (lamda - C4), [x0, x, y], g3, [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
            end
        end
    end
        
    optimize!(model) # solve for coefficients
    status = termination_status(model)
    C_eval_dict = Dict() # Get numerical values of coefficients and plug into polynomials
    for (key, (C, Cc, Cb)) in C_dict
        coeff_vals = round.(value.(Cc); digits=error)  # Round each coefficient to 2 decimal places
        C_numeric = sum(coeff_vals[i] * Cb[i] for i in eachindex(Cb))
        C_eval_dict[key] = (C, C_numeric)
    end
    
    # status might be optimal but if all Bc approx 10^{-error}, it's essentially 0 so OPTIMAL != CC.
    return status, C_eval_dict
end

# the considered NBA 

# ~A representing ~(b U a) = (~b R ~a)
S1 = Dict(
    :"q0" => [(Set(["b"]), :"q0"), (Set(["a"]), :"q1")],
    :"q1" => [(Set(["a", "b"]), :"q1")]
)

Q01, QF1 = Set(["q0"]), Set(["q0"]) # Initial and Acceptance set of ~NBA

# Simulation
file = open("./systems/BecomingRichOnce_SCC_SOS.txt", "w")
deg = 1 # desired degree of SCC
write(file, "poly deg: "*string(deg)*"\n")

stats = @timed (status, CC_data) = scc(deg, S1, Q01, QF1)

write(file, "status: "*string(status)*"\n")
write(file, "Number of SCC polynomials: "*string(length(CC_data))*"\n")
write(file, "time: "*string(stats.time)*"\n\n")

# Write the dictionary line-by-line
for (k, v) in CC_data
    write(file, "Key: $(k)\n")
    write(file, "Polynomial: $(v[1])\n")
    write(file, "Coefficients: $(v[2])\n\n")
end

close(file)

println("Finished")

Finished
